In [1]:
#parse name, cabin into separate columns

In [2]:
from tensorflow import keras
from keras import layers
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer

In [3]:
def od_ordinal(object_cols, *X_tuple):
    """Takes in X dataframe(s) and a list of column names with object
       data type and returns an ordinal encoded X dataframe(s)."""

    ordinal = OrdinalEncoder()
    
    for x in X_tuple:
        x[object_cols] = pd.DataFrame(ordinal.fit_transform(x[object_cols]))
        
    return X_tuple

def od_imputer(numerical_cols, object_cols, *X_tuple):
    """Imputes the numerical and categorical columns of X dataframe(s) before returning the
       said dataframe(s)."""

    imputer_mean = SimpleImputer()
    imputer_mode = SimpleImputer(strategy='most_frequent')

    for x in X_tuple:
        x[numerical_cols] = imputer_mean.fit_transform(x[numerical_cols])
        x[object_cols] = imputer_mode.fit_transform(x[object_cols])
    
    return X_tuple

In [4]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

X = train_data.drop(['PassengerId', 'Transported'], axis=1)
y = train_data.Transported
X_test = test_data.drop('PassengerId', axis=1)

# parsing Cabin and Name columns
X[['Cabin', 'Cabin2', 'Cabin3']] = X['Cabin'].str.split('/', expand=True)
X[['First Name', 'Last Name']] = X['Name'].str.split(' ', expand=True)
X_test[['Cabin', 'Cabin2', 'Cabin3']] = X_test['Cabin'].str.split('/', expand=True)
X_test[['First Name', 'Last Name']] = X_test['Name'].str.split(' ', expand=True)
X['Cabin2'] = X['Cabin2'].astype('float')
X_test['Cabin2'] = X_test['Cabin2'].astype('float')

object_cols = [col for col in X.columns if X[col].dtype=='object']
numerical_cols = list(set(X.columns) - set(object_cols))

# preprocessing
X, X_test = od_ordinal(object_cols, X, X_test)
X, X_test = od_imputer(numerical_cols, object_cols, X, X_test)

In [5]:
model = keras.Sequential([layers.BatchNormalization(input_shape=[16]),
                         layers.Dense(1024, activation='relu'),
                         layers.BatchNormalization(),
                         layers.Dropout(rate=0.5),
                         layers.Dense(512, activation='relu'),
                         layers.BatchNormalization(),
                         layers.Dropout(rate=0.5),
                         layers.Dense(256, activation='relu'),
                         layers.BatchNormalization(),
                          layers.Dropout(rate=0.5),
                         layers.Dense(128, activation='relu'),
                          layers.BatchNormalization(),
                          layers.Dropout(rate=0.5),
                          layers.Dense(64, activation='relu'),
                          layers.BatchNormalization(),
                          layers.Dropout(rate=0.5),
                          layers.Dense(1, activation='sigmoid')
                         ])

model.compile(loss=keras.losses.BinaryCrossentropy(),
              optimizer=keras.optimizers.Adam(),
              metrics=['binary_accuracy']
)

fit_object = model.fit(X, y,
                       epochs=100,
                       batch_size=128,
                       verbose=1
         )

Epoch 1/100
68/68 [==============================] - 9s 81ms/step - loss: 0.7610 - binary_accuracy: 0.6935
Epoch 2/100
68/68 [==============================] - 6s 87ms/step - loss: 0.5374 - binary_accuracy: 0.7531
Epoch 3/100
68/68 [==============================] - 5s 78ms/step - loss: 0.4945 - binary_accuracy: 0.7697
Epoch 4/100
68/68 [==============================] - 5s 77ms/step - loss: 0.4679 - binary_accuracy: 0.7776
Epoch 5/100
68/68 [==============================] - 5s 76ms/step - loss: 0.4582 - binary_accuracy: 0.7802
Epoch 6/100
68/68 [==============================] - 5s 78ms/step - loss: 0.4526 - binary_accuracy: 0.7833
Epoch 7/100
68/68 [==============================] - 6s 85ms/step - loss: 0.4444 - binary_accuracy: 0.7929
Epoch 8/100
68/68 [==============================] - 6s 84ms/step - loss: 0.4420 - binary_accuracy: 0.7887
Epoch 9/100
68/68 [==============================] - 6s 82ms/step - loss: 0.4359 - binary_accuracy: 0.7912
Epoch 10/100
68/68 [=================

In [6]:
yhat_test = np.where(model.predict(X_test).squeeze()>=0.5, True, False)

output = pd.DataFrame({'PassengerId': test_data.PassengerId,
                       'Transported': yhat_test
})

output.to_csv('submission_nn.csv', index=False)

134/134 [==============================] - 1s 9ms/step
